In [11]:
import os
import sys
import pandas as pd

In [12]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [20]:
import src

## Создание датасета для дальнейшей обработки

[Исходные данные](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction)

Чтение исходных данных:

In [13]:
application_data = pd.read_csv("../data/raw/application_record.csv")
credit_data = pd.read_csv("../data/raw/credit_record.csv")

In [14]:
application_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AMT_INCOME_TOTAL
0,5008804,M,Y,Y,0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,436565.0
1,5008805,M,Y,Y,0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,436565.0
2,5008806,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,115180.0
3,5008808,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,265080.0
4,5008809,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,265080.0


In [7]:
credit_data.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


Описание исходных данных:
*application_record* (записи кредитных заявок):
- *ID* - идентификатор заявки;
- *CODE_GENDER* - пол;
- *FLAG_OWN_CAR* - наличие автомобиля;
- *FLAG_OWN_REALTY* - наличие недвижимости;
- *CNT_CHILDREN* - число детей;
- *AMT_INCOME_TOTAL* - годовой доход;
- *NAME_INCOME_TYPE* - тип дохода;
- *NAME_EDUCATION_TYPE* - образование;
- *NAME_FAMILY_STATUS* - семейное положение;
- *NAME_HOUSING_TYPE* - тип жилья;
- *DAYS_BIRTH* - возраст в днях (считается в обратном порядке от текущего дня, например, вчерашний день записывается как -1);
- *DAYS_EMPLOYED* - стаж на текущей работе в днях (считается в обратном порядке, положительное значение означает отсутствие работы);
- *FLAG_MOBIL* - наличие мобильного телефона;
- *FLAG_WORK_PHONE* - наличие рабочего телефона;
- *FLAG_PHONE* - наличие телефона;
- *FLAG_EMAIL* - наличие электронной почты;
- *OCCUPATION_TYPE* - тип занятости;
- *CNT_FAMILY_MEMBERS* - число членов семьи;

*credit_record* (записи кредитных историй):
- *ID* - Идентификатор кредитной заявки;
- *MONTHS_BALANCE* - число месяцев, прошедших с текущей даты (считается в обратном порядке);
- *STATUS* - статус: 0: просроченный платеж на 1-29 дней; 1 - на 30-59 дней; 2 - на 60-89 дней; 3 - на 90-119 дней; 4 - на 120-149 дней; 5 - 150 дней и более; C - долги выплачены в текущем месяце; X - нет долгов на текущий месяц.

Для получения первоначального датасета для дальнешей предобработки наобходимо совершить ряд преобразований:
1) В исходных данных отсутствуют метки классов. Исходная задача состоит в поиске "плохих" клиентов, будем считать клиента "плохим" и присваивать ему метку класса 1, если у него в каком-либо месяце есть просрочка кредитного платежа на 90 дней или более (или иными словами, поле *STATUS* принимает значения 3, 4 или 5);

2) В данных присутствуют идентификаторы кредитных заявок, но отсутствуют идентификаторы самих клиентов, несмотря на то, что один клиент может иметь несколько кредитов. Это можно увидеть по наличию дубликатов в данных о заявках с выброшенным столбцом *ID*: 

In [18]:
application_data.drop(["ID"], axis=1).duplicated().sum()

348472

Необходимо сгруппировать данные по клиентам.

Описанные выше преобразования выполняются в скрипте *src/data/make_dataset.py*. Полученный датасет записывается в файл *cleaned_dataset.csv*.

In [24]:
initial_data = pd.read_csv("../data/interim/initial_dataset.csv")

In [26]:
initial_data.head()

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AMT_INCOME_TOTAL,CLIENT_ID,BAD_CLIENT
0,M,Y,Y,0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,436565.0,0,0
1,M,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,115180.0,1,0
2,F,N,Y,0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,265080.0,2,0
3,F,N,Y,0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0,283695.0,3,0
4,M,Y,Y,0,Working,Higher education,Married,House / apartment,-16872,-769,1,1,1,1,Accountants,2.0,266667.0,4,0


Полученные данные не имеют дубликатов, но имеют пропуски в столбце "OCCUPATION_TYPE":

In [27]:
initial_data.isna().sum()

CODE_GENDER               0
FLAG_OWN_CAR              0
FLAG_OWN_REALTY           0
CNT_CHILDREN              0
NAME_INCOME_TYPE          0
NAME_EDUCATION_TYPE       0
NAME_FAMILY_STATUS        0
NAME_HOUSING_TYPE         0
DAYS_BIRTH                0
DAYS_EMPLOYED             0
FLAG_MOBIL                0
FLAG_WORK_PHONE           0
FLAG_PHONE                0
FLAG_EMAIL                0
OCCUPATION_TYPE        3001
CNT_FAM_MEMBERS           0
AMT_INCOME_TOTAL          0
CLIENT_ID                 0
BAD_CLIENT                0
dtype: int64

Часть работников не имеют типа занятости. Логично предположить, что часть клиентов могут не иметь типа занятости из-за отсутствия работы:

In [29]:
initial_data[initial_data['DAYS_EMPLOYED'] > 0].shape[0]

1699

In [31]:
initial_data[initial_data['DAYS_EMPLOYED'] > 0].isna().sum()['OCCUPATION_TYPE']

1699

Действительно, все безработные клиенты имеют пропущенное значение в столбце *OCCUPATION_TYPE*. Будем заполнять такие пропуски значением *Unemployed*. Пропуски у занятых клиентов будем заполнять значением *Other occupations*.

Помимо этого, в датасете присутствует признак *FLAG_MOBIL* с нулевой дисперсией, который можно удалить:

In [34]:
initial_data['FLAG_MOBIL'].nunique()

1

Очистка данных производится в скрипте *src/data/clean_data.py* и, помимо описанных выше процедур, включает в себя перевод возраста и стажа из дней в годы, приведение значений бинарных признаков к одному виду и изменение порядка столбцов в таблице. 

In [35]:
cleaned_data = pd.read_csv('../data/interim/cleaned_dataset.csv')

In [36]:
cleaned_data.head()

,YEARS_BIRTH,CODE_GENDER,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,YEARS_EMPLOYED,OCCUPATION_TYPE,NAME_EDUCATION_TYPE,CNT_FAM_MEMBERS,CNT_CHILDREN,NAME_FAMILY_STATUS,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_HOUSING_TYPE,FLAG_PHONE,FLAG_WORK_PHONE,FLAG_EMAIL,BAD_CLIENT
0,32.868574,M,436565.0,Working,12.435574,Other Occupations,Higher education,2,0,Civil marriage,No,No,Rented apartment,No,Yes,No,0
1,58.793815,M,115180.0,Working,3.104787,Security staff,Secondary / secondary special,2,0,Married,No,No,House / apartment,No,No,No,0
2,52.321403,F,265080.0,Commercial associate,8.353354,Sales staff,Secondary / secondary special,1,0,Single / not married,No,No,House / apartment,Yes,No,Yes,0
3,61.504343,F,283695.0,Pensioner,-0.002738,Unemployed,Higher education,1,0,Separated,No,No,House / apartment,No,No,No,0
4,46.193967,M,266667.0,Working,2.105450,Accountants,Higher education,2,0,Married,No,No,House / apartment,Yes,Yes,Yes,0
